In [ ]:
# Baseline Models: Results and Explainability

This notebook summarizes baseline training/evaluation and SHAP explainability using the sample datasets.



In [ ]:
import json
from pathlib import Path

import joblib
import pandas as pd

from healthai.src.models.tabular.evaluate import load_xy
from healthai.src.utils.metrics import classification_metrics, regression_metrics

DATA_CLS = Path('healthai/data/raw/classification_breast_cancer.csv')
DATA_REG = Path('healthai/data/raw/regression_diabetes.csv')
MODEL_CLS = Path('healthai/models/baseline_classification.pkl')
MODEL_REG = Path('healthai/models/baseline_regression.pkl')

def report(model_path: Path, csv: Path, target: str, task: str):
    model = joblib.load(model_path)
    X, y = load_xy(csv, target)
    if task == 'classification':
        y_pred = model.predict(X)
        try:
            y_proba = model.predict_proba(X)[:, 1]
        except Exception:
            y_proba = None
        return classification_metrics(y, y_pred, y_proba)
    else:
        y_pred = model.predict(X)
        return regression_metrics(y, y_pred)

print('Classification metrics:', report(MODEL_CLS, DATA_CLS, 'target', 'classification'))
print('Regression metrics:', report(MODEL_REG, DATA_REG, 'target', 'regression'))


In [ ]:
from pathlib import Path
from healthai.src.models.tabular.shap_explain import load_xy
import joblib

csv = Path('healthai/data/raw/classification_breast_cancer.csv')
model_path = Path('healthai/models/baseline_classification.pkl')
X, y = load_xy(csv, 'target')
pipe = joblib.load(model_path)

# Display top features using model's feature_importances_ if available
est = getattr(pipe, 'named_steps', {}).get('model', pipe)
if hasattr(est, 'feature_importances_'):
    import numpy as np
    importances = est.feature_importances_
    # Feature names not directly available post-encoding; list top indices
    top_idx = np.argsort(importances)[-10:][::-1]
    print('Top feature indices by importance:', top_idx)
else:
    print('Model has no feature_importances_ (skip)')
